In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from ipywidgets import interact, Dropdown
from IPython.display import display


In [3]:
df_co2 = pd.read_csv('data/co2.csv')
df_co2['year'] = df_co2['year'].astype(int)
country_options = sorted(df_co2['country_code'].unique())
print(f'Loaded {len(df_co2)} rows for {len(country_options)} countries.')


Loaded 9466 rows for 206 countries.


In [ ]:
def plot_hw_forecast(country_code):
    country_data = df_co2[df_co2['country_code'] == country_code].sort_values('year')
    if country_data.empty:
        raise ValueError(f'No data found for {country_code}')


    forecast_steps = 6
    forecast_years = np.arange(2020, 2020 + forecast_steps)

    
    train = country_data[(country_data['year'] >= 1970) & (country_data['year'] <= 2019)]
    if len(train) < 8:
        return pd.Series(np.nan, index=forecast_years)
    ts = train['co2'].values
    model = ExponentialSmoothing(ts, trend='add', seasonal=None)
    fitted = model.fit(optimized=True)

    
    forecast = fitted.forecast(steps=forecast_steps)
    forecast = np.asarray(forecast)

    # Forecast years should start at 2020
    

    # Optional: force first forecast point to equal last observed point
    forecast[0] = ts[-1]

    return pd.Series(forecast, index=forecast_years)

predictions = {
    c: plot_hw_forecast(c)
    for c in df_co2['country_code'].unique()
}

predictions = pd.DataFrame(predictions)


UnboundLocalError: cannot access local variable 'forecast_years' where it is not associated with a value

KeyError: 'country_code'